<a href="https://colab.research.google.com/github/elvingeorgee/Autoboxing/blob/main/NBA_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install nba.api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.2/257.2 KB 5.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import requests
import nba_api as nba
from nba_api.stats import endpoints

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [3]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
MVP = pd.read_csv('/content/drive/MyDrive/Religion/mvp_votes.csv').drop('Unnamed: 0', axis = 1)
MVP = MVP[['player','season','points_won','votes_first']]

In [5]:
display(MVP)

,player,season,points_won,votes_first
0,Julius Erving,1980-81,454.0,28.0
1,Larry Bird,1980-81,423.0,20.0
2,Kareem Abdul-Jabbar,1980-81,286.0,8.0
3,Moses Malone,1980-81,180.0,8.0
4,George Gervin,1980-81,83.0,1.0
...,...,...,...,...
632,LaMarcus Aldridge,2017-18,6.0,0.0
633,Jimmy Butler,2017-18,5.0,0.0
634,Stephen Curry,2017-18,5.0,0.0
635,Joel Embiid,2017-18,4.0,0.0


In [6]:
MVP_points = MVP.groupby(['season'])[['points_won']].max().reset_index()
MVP = MVP.merge(MVP_points, on = 'season', how = 'left')

In [7]:
def is_MVP(points, points_winner):
  if points == points_winner: 
    return 1
  else:
    return 0

In [8]:
MVP['MVP'] = MVP.apply(lambda x: is_MVP(x.points_won_x, x.points_won_y), axis = 1)

In [9]:
MVP = MVP.drop(['points_won_y'], axis = 1)
MVP.head()

,player,season,points_won_x,votes_first,MVP
0,Julius Erving,1980-81,454.0,28.0,1
1,Larry Bird,1980-81,423.0,20.0,0
2,Kareem Abdul-Jabbar,1980-81,286.0,8.0,0
3,Moses Malone,1980-81,180.0,8.0,0
4,George Gervin,1980-81,83.0,1.0,0


In [10]:
seasons = MVP['season'].unique()

In [11]:
seasons

array(['1980-81', '1981-82', '1982-83', '1983-84', '1984-85', '1985-86',
       '1986-87', '1987-88', '1988-89', '1989-90', '1990-91', '1991-92',
       '1992-93', '1993-94', '1994-95', '1995-96', '1996-97', '1997-98',
       '1998-99', '1999-00', '2000-01', '2001-02', '2002-03', '2003-04',
       '2004-05', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10',
       '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16',
       '2016-17', '2017-18'], dtype=object)

In [12]:
data = endpoints.leagueleaders.LeagueLeaders(league_id = '00',
                                             per_mode48 = 'Totals', 
                                             scope = 'S',
                                             season = "1979-80",
                                             season_type_all_star = 'Regular Season').league_leaders.get_data_frame()

data['season'] = '1979-80'
for i in seasons:
    temp = endpoints.leagueleaders.LeagueLeaders(league_id = '00',
                                                per_mode48 = 'Totals',
                                                scope = 'S',
                                                season = i,
                                                season_type_all_star = 'Regular Season').league_leaders.get_data_frame()
    temp['season'] = i
    data = pd.concat([data, temp])

In [13]:
MVP = MVP.rename(columns={"player": "PLAYER"})
df = data.merge(MVP, how = 'left', on = ['season','PLAYER'])
df = df[df['season']!='1979-80']
df['MVP'] = df['MVP'].fillna(0)

In [14]:
X = df.drop(['MVP','PLAYER','RANK','votes_first','points_won_x','TEAM','PLAYER_ID'],axis = 1)
y  = df['MVP']

X = X.groupby('season').transform(lambda x: (x - x.mean()) / x.std())

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

clf = LogisticRegression(random_state=0).fit(X_train, y_train)

clf.score(X_train, y_train)

0.9977190648165748

In [16]:
clf.score(X_test, y_test)

0.9982635539262975

In [17]:
df['predicted_MVP'] = clf.predict(X)

df['predicted_MVP_prob'] = clf.predict_proba(X)[:,1:]

df[df['season']=='2015-16'].sort_values(by='predicted_MVP_prob',ascending=False).head(5)

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV,season,points_won_x,votes_first,MVP,predicted_MVP,predicted_MVP_prob
14461,201939,2,Stephen Curry,1610612744,GSW,79,2700,805,1598,0.504,402,886,0.454,363,400,0.908,68,362,430,527,169,15,262,161,2375,2424,2.01,0.65,2015-16,1310.0,131.0,1.0,1.0,0.723646
14462,201142,3,Kevin Durant,1610612760,OKC,72,2578,698,1381,0.505,186,481,0.387,447,498,0.898,45,544,589,361,69,85,250,137,2029,2149,1.44,0.28,2015-16,147.0,0.0,0.0,0.0,0.314613
14465,201566,6,Russell Westbrook,1610612760,OKC,80,2750,656,1444,0.454,101,341,0.296,465,573,0.812,145,481,626,834,163,20,342,200,1878,2283,2.44,0.48,2015-16,486.0,0.0,0.0,0.0,0.222061
14463,2544,4,LeBron James,1610612739,CLE,76,2709,737,1416,0.520,87,282,0.309,359,491,0.731,111,454,565,514,104,49,249,143,1920,2092,2.06,0.42,2015-16,631.0,0.0,0.0,0.0,0.159142
14460,201935,1,James Harden,1610612745,HOU,82,3125,710,1617,0.439,236,657,0.359,720,837,0.860,63,438,501,612,139,51,374,229,2376,2281,1.64,0.37,2015-16,9.0,0.0,0.0,0.0,0.068717


In [18]:
df[df['season']=='2014-15'].sort_values(by='predicted_MVP_prob',ascending=False).head(5)

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV,season,points_won_x,votes_first,MVP,predicted_MVP,predicted_MVP_prob
13969,201939,2,Stephen Curry,1610612744,GSW,80,2613,653,1341,0.487,286,646,0.443,308,337,0.914,56,285,341,619,163,16,249,158,1900,2073,2.49,0.66,2014-15,1198.0,100.0,1.0,0.0,0.358914
13975,203076,8,Anthony Davis,1610612740,NOP,68,2455,642,1199,0.535,1,12,0.083,371,461,0.805,173,523,696,149,100,200,95,141,1656,2059,1.57,1.05,2014-15,203.0,0.0,0.0,0.0,0.277104
13977,101108,10,Chris Paul,1610612746,LAC,82,2857,568,1170,0.485,139,349,0.398,289,321,0.900,52,324,376,838,156,15,190,203,1564,2125,4.41,0.82,2014-15,124.0,0.0,0.0,0.0,0.204668
13968,201935,1,James Harden,1610612745,HOU,81,2981,647,1470,0.440,208,555,0.375,715,824,0.868,75,384,459,565,154,60,321,208,2217,2202,1.76,0.48,2014-15,936.0,25.0,0.0,0.0,0.118150
13970,201566,3,Russell Westbrook,1610612760,OKC,67,2302,627,1471,0.426,86,288,0.299,546,654,0.835,124,364,488,574,140,14,293,184,1886,1857,1.96,0.48,2014-15,352.0,0.0,0.0,0.0,0.082703


In [19]:
prob_MVP = df.groupby(['season'])[['predicted_MVP_prob']].max().reset_index()

df = df.merge(prob_MVP, on = 'season', how = 'left')

def is_MVP_prob(prob, prob_winner):
    if prob == prob_winner:
        return 1
    else:
        return 0

df['MVP_prob'] = df.apply(lambda x: is_MVP_prob(x.predicted_MVP_prob_x, x.predicted_MVP_prob_y), axis=1)

In [20]:
df[df['season']=='2014-15'].sort_values(by='predicted_MVP_prob_x',ascending=False).head(5)


,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV,season,points_won_x,votes_first,MVP,predicted_MVP,predicted_MVP_prob_x,predicted_MVP_prob_y,MVP_prob
13712,201939,2,Stephen Curry,1610612744,GSW,80,2613,653,1341,0.487,286,646,0.443,308,337,0.914,56,285,341,619,163,16,249,158,1900,2073,2.49,0.66,2014-15,1198.0,100.0,1.0,0.0,0.358914,0.358914,1
13718,203076,8,Anthony Davis,1610612740,NOP,68,2455,642,1199,0.535,1,12,0.083,371,461,0.805,173,523,696,149,100,200,95,141,1656,2059,1.57,1.05,2014-15,203.0,0.0,0.0,0.0,0.277104,0.358914,0
13720,101108,10,Chris Paul,1610612746,LAC,82,2857,568,1170,0.485,139,349,0.398,289,321,0.900,52,324,376,838,156,15,190,203,1564,2125,4.41,0.82,2014-15,124.0,0.0,0.0,0.0,0.204668,0.358914,0
13711,201935,1,James Harden,1610612745,HOU,81,2981,647,1470,0.440,208,555,0.375,715,824,0.868,75,384,459,565,154,60,321,208,2217,2202,1.76,0.48,2014-15,936.0,25.0,0.0,0.0,0.118150,0.358914,0
13713,201566,3,Russell Westbrook,1610612760,OKC,67,2302,627,1471,0.426,86,288,0.299,546,654,0.835,124,364,488,574,140,14,293,184,1886,1857,1.96,0.48,2014-15,352.0,0.0,0.0,0.0,0.082703,0.358914,0


In [21]:
from sklearn.metrics import classification_report
print(classification_report(df['MVP'], df['MVP_prob']))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     15667
         1.0       0.58      0.58      0.58        38

    accuracy                           1.00     15705
   macro avg       0.79      0.79      0.79     15705
weighted avg       1.00      1.00      1.00     15705



In [22]:
df["rank"] = df.groupby("season")["predicted_MVP_prob_x"].rank("dense", ascending=False)

MVP_rank = df[['points_won_x','votes_first','MVP','rank']]
MVP_rank.head()

,points_won_x,votes_first,MVP,rank
0,15.0,1.0,0.0,4.0
1,180.0,8.0,0.0,1.0
2,83.0,1.0,0.0,6.0
3,286.0,8.0,0.0,2.0
4,454.0,28.0,1.0,3.0


In [23]:
MVP_first = MVP_rank[MVP_rank['MVP']==1]
MVP_first['difference'] = MVP_first['rank']-1

MVP_first

,points_won_x,votes_first,MVP,rank,difference
4,454.0,28.0,1.0,3.0,2.0
279,507.0,40.0,1.0,1.0,0.0
579,720.0,69.0,1.0,4.0,3.0
896,652.0,52.0,1.0,1.0,0.0
1200,763.0,73.0,1.0,1.0,0.0
1511,765.0,73.0,1.0,1.0,0.0
1830,733.0,65.0,1.0,1.0,0.0
2157,665.0,47.0,1.0,2.0,1.0
2503,664.5,42.5,1.0,2.0,1.0
2857,636.0,27.0,1.0,2.0,1.0


In [24]:
MVP_first['difference'].mean()


1.105263157894737

In [25]:
season_2000 = df[df['season']=='2000-01']
season_2000.sort_values(by='rank').head(12)


,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV,season,points_won_x,votes_first,MVP,predicted_MVP,predicted_MVP_prob_x,predicted_MVP_prob_y,MVP_prob,rank
7379,708,15,Kevin Garnett,1610612750,MIN,81,3202,704,1475,0.477,19,66,0.288,357,467,0.764,219,702,921,401,111,145,230,204,1784,2251,1.74,0.48,2000-01,151.0,1.0,0.0,0.0,0.109760,0.10976,1,1.0
7369,1503,6,Tracy McGrady,1610612753,ORL,77,3087,788,1724,0.457,59,166,0.355,430,587,0.733,192,388,580,352,116,118,198,160,2065,1940,1.78,0.59,2000-01,64.0,0.0,0.0,0.0,0.100715,0.10976,0,2.0
7366,406,3,Shaquille O'Neal,1610612747,LAL,74,2924,813,1422,0.572,0,2,0.000,499,972,0.513,291,649,940,277,47,204,218,256,2125,2293,1.27,0.22,2000-01,578.0,7.0,0.0,0.0,0.051797,0.10976,0,3.0
7364,711,1,Jerry Stackhouse,1610612765,DET,80,3215,774,1927,0.402,166,473,0.351,666,810,0.822,99,216,315,410,97,54,326,160,2380,1633,1.26,0.30,2000-01,3.0,0.0,0.0,0.0,0.039977,0.10976,0,4.0
7375,56,12,Gary Payton,1610612760,SEA,79,3244,725,1591,0.456,102,272,0.375,271,354,0.766,73,288,361,642,127,26,209,184,1823,1821,3.07,0.61,2000-01,NaN,NaN,0.0,0.0,0.034672,0.10976,0,5.0
7372,185,9,Chris Webber,1610612758,SAC,70,2836,786,1635,0.481,2,28,0.071,324,461,0.703,179,598,777,294,93,118,195,226,1898,1999,1.51,0.48,2000-01,521.0,5.0,0.0,0.0,0.032142,0.10976,0,6.0
7368,1713,5,Vince Carter,1610612761,TOR,75,2979,762,1656,0.460,162,397,0.408,384,502,0.765,176,240,416,291,114,82,167,205,2070,1794,1.74,0.68,2000-01,7.0,0.0,0.0,0.0,0.026173,0.10976,0,7.0
7377,951,14,Ray Allen,1610612749,MIL,82,3129,628,1309,0.480,202,467,0.433,348,392,0.888,101,327,428,374,124,20,204,192,1806,1823,1.83,0.61,2000-01,7.0,0.0,0.0,0.0,0.016657,0.10976,0,8.0
7374,252,11,Karl Malone,1610612762,UTA,81,2895,670,1345,0.498,2,5,0.400,536,676,0.793,114,555,669,361,93,62,244,216,1878,2004,1.48,0.38,2000-01,21.0,0.0,0.0,0.0,0.016465,0.10976,0,9.0
7376,1495,13,Tim Duncan,1610612759,SAS,82,3174,702,1406,0.499,7,27,0.259,409,662,0.618,259,738,997,245,70,192,242,247,1820,2125,1.01,0.29,2000-01,706.0,18.0,0.0,0.0,0.011927,0.10976,0,10.0
